<h1><span style="color:red">NEMO Annotation for SuAVE</span></h1>

This notebook uses the NEMO (Named Entities Made Obvious) service developed at Microsoft Research (author: Dr. Silviu Cuzerzan, see https://doi.org/10.1145/2633211.2634360) to generate a set of named entities for user's text


## 1. Retrieve survey parameters from the URL

In [1]:
%%javascript
function getQueryStringValue (key)
{  
    return unescape(window.location.search.replace(new RegExp("^(?:.*[&\\?]" + escape(key).replace(/[\.\+\*]/g, "\\$&") + "(?:\\=([^&]*))?)?.*$", "i"), "$1"));
}
IPython.notebook.kernel.execute("survey_url='".concat(getQueryStringValue("surveyurl")).concat("'"));
IPython.notebook.kernel.execute("views='".concat(getQueryStringValue("views")).concat("'"));
IPython.notebook.kernel.execute("view='".concat(getQueryStringValue("view")).concat("'"));
IPython.notebook.kernel.execute("user='".concat(getQueryStringValue("user")).concat("'"));
IPython.notebook.kernel.execute("csv_file='".concat(getQueryStringValue("csv")).concat("'")); 
IPython.notebook.kernel.execute("dzc_file='".concat(getQueryStringValue("dzc")).concat("'")); 
IPython.notebook.kernel.execute("params='".concat(getQueryStringValue("params")).concat("'")); 
IPython.notebook.kernel.execute("active_object='".concat(getQueryStringValue("activeobject")).concat("'")); 
IPython.notebook.kernel.execute("full_notebook_url='" + window.location + "'"); 

<IPython.core.display.Javascript object>

## 2. Setting up the environment, importing libraries, defining functions

In [2]:
# common imports
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import Markdown, display

import pandas as pd
pd.set_option('display.max_colwidth', 0)
    
import numpy as np
import panel as pn

pn.extension()
def printmd(string):
    display(Markdown(string))

absolutePath = "/home/jovyan/temp_csvs/"

# local imports
import sys
sys.path.insert(1, '../../helpers')
import panel_libs as panellibs
import suave_integration as suaveint


# specific imports
import requests
from urllib.parse import urlparse
import re
import nemofunc as nemo


In [3]:
%%capture
from tqdm import tqdm_notebook as tqdm
tqdm().pandas()

## 3. Select a survey file from SuAVE or import a local CSV file

In [4]:
data_select = pn.widgets.RadioBoxGroup(name='Select notebook', options=['Load survey file from SuAVE', 
                                                                        'Import a local CSV file'], 
                                       inline=False)
data_select

RadioBoxGroup(name='Select notebook', options=['Load survey file from Su...], value='Load survey f...)

In [5]:
data_input = pn.widgets.FileInput()
    
def check_selection():
    if data_select.value == 'Load survey file from SuAVE':
        global fname
        fname = absolutePath + csv_file
        printmd("<b><span style='color:red; font-size: 200%;'>Current SuAVE survey will be loaded. Continue to step 4.</span></b>")

    else:
        message = pn.pane.HTML("<b><span style='color:red; font-size: 200%;'>Upload data and continue to step 4.</span><br><span style='font-size: 150%;'>IMPORTANT: The local CSV file should not have SuAVE-specific variable names!</span></b>", width=700)
        return pn.Column(message, data_input)
    
check_selection()

<b><span style='color:red; font-size: 200%;'>Current SuAVE survey will be loaded. Continue to step 4.</span></b>

## 4. Visualize the data and select a text variable to parse

In [6]:
if not pd.isnull(data_input.filename):
    fname = absolutePath + data_input.filename
    data_input.save(fname)

df = panellibs.extract_data(fname)
panellibs.slider(df)


Column(width=800)
    [0] Row
        [0] IntSlider(end=1376, margin=(0, 50, -15, 0), name='Navigate Rows', width=350)
        [1] IntSlider(end=16, margin=(0, 0, 5, 0), name='Navigate Columns', width=350)
    [1] ParamFunction(function, _pane=DataFrame, defer_load=False, width=800)

In [7]:
# if the above didn't display the dataframe:
with pd.option_context("display.max_columns", None):
    if any("geometry" in col for col in df.columns):
        display(df.drop(['geometry'],axis=1))
    else:
        display(df)
    

,Name,OAID#link#multi,Affiliation#sortquan,City#sortquan,Region#sortquan,Country#sortquan,Latitude#hidden,Longitude#hidden,Collaborators#multi#link#sortquan,Scope#multi#sortquan,Keywords#multi#sortquan,OA concepts#multi#sortquan,Publications#hidden,Publication Dates#multi#sortquan,#img,#netvis
0,A Olioso,https://openalex.org/A4227955457,Unknown,NaN,NaN,NaN,NaN,NaN,https://openalex.org/A4227955454|https://openalex.org/A4227955461|https://openalex.org/A4227955455|https://openalex.org/A4227955463|https://openalex.org/A4227955453|https://openalex.org/A4227955464|https://openalex.org/A4227955456|https://openalex.org/A4227955462|https://openalex.org/A4227955460|https://openalex.org/A4227955459|https://openalex.org/A4227955452|https://openalex.org/A4227955458,aquifer|transboundary,NaN,Groundwater|Geology|Geotechnical engineering|Hydrology (agriculture)|Aquifer|Environmental science|Computer science|Water resource management,"<a href='#' onClick='javascript:getPublication({oaids:""https://openalex.org/A4227955457"",search:""Keywords,Scope"",OAConcepts:""OA concepts""})'>Show publications</a>",2021,US,02ac504b6e11517e2110d174ea70a1a7ac1cf19899e1a0f23c29558f6225db03
1,A Olioso,https://openalex.org/A4226682424,Unknown,NaN,NaN,NaN,NaN,NaN,https://openalex.org/A4226682420|https://openalex.org/A4226682425|https://openalex.org/A4226682421|https://openalex.org/A4226682429|https://openalex.org/A4226682431|https://openalex.org/A4226682426|https://openalex.org/A4226682422|https://openalex.org/A4226682428|https://openalex.org/A4226682427|https://openalex.org/A4226682419|https://openalex.org/A4226682423|https://openalex.org/A4226682430,aquifer|transboundary,NaN,Groundwater|Geology|Geotechnical engineering|Hydrology (agriculture)|Aquifer|Environmental science|Water resource management,"<a href='#' onClick='javascript:getPublication({oaids:""https://openalex.org/A4226682424"",search:""Keywords,Scope"",OAConcepts:""OA concepts""})'>Show publications</a>",2021,US,8f95a1d08aacc416f1abe22426fe9c9fd2f8f338bb7365407f284e3985165d23
2,A. Alassane,https://openalex.org/A2484425674,Cheikh Anta Diop University,Dakar,NaN,Senegal,14.686944,-17.463333,https://openalex.org/A2434763705|https://openalex.org/A3069707669|https://openalex.org/A2182351332|https://openalex.org/A3051995119,aquifer|transboundary,NaN,Sociology|Population|Water supply|Demography|Groundwater|Water quality|Ecology|Geology|Geotechnical engineering|Groundwater recharge|Hydrology (agriculture)|Environmental engineering|Aquifer|Biology|Environmental science|Water resource management,"<a href='#' onClick='javascript:getPublication({oaids:""https://openalex.org/A2484425674"",search:""Keywords,Scope"",OAConcepts:""OA concepts""})'>Show publications</a>",2010,US,f55f8f5c25002f0f2a2e121be602248623f07494d5161a13d399ab12aa746bac
3,A. Aureli,https://openalex.org/A2422334401,Unknown,NaN,NaN,NaN,NaN,NaN,https://openalex.org/A2304341794|https://openalex.org/A2182540860,aquifer|transboundary,NaN,Karst|Biology|Tourism|Business|Environmental planning|Archaeology|Environmental science|Groundwater|Geotechnical engineering|Water resources|Water resource management|Environmental resource management|Environmental protection|Law|Ecology|Engineering|Multidisciplinary approach|Aquifer|Geography|Political science,"<a href='#' onClick='javascript:getPublication({oaids:""https://openalex.org/A2422334401"",search:""Keywords,Scope"",OAConcepts:""OA concepts""})'>Show publications</a>",2010,US,fee72a7c6e6595abd9a1fe8878cb3c9be76652d50b3986c6b1cb4ac610869e76
4,A. Aureli,https://openalex.org/A3086349667,Unknown,NaN,NaN,NaN,NaN,NaN,https://openalex.org/A3085518772|https://openalex.org/A3085940897|https://openalex.org/A3086175637|https://openalex.org/A3086707070|https://openalex.org/A3216340081|https://openalex.org/A3084770820|https://openalex.org/A3085504345,aquifer|transboundary,NaN,Environmental resource management|Groundwater|Environmental planning|Geology|Geotechnical engineering|Hydrology (agriculture)|Aquifer|Environmental

## 5. Generate pre-defined #multi variables for entity types extracted by NEMO

In [8]:
varcols = df.columns.tolist()
# remove any variable names are unlikely to contain parsable text 
varcols = [x for x in varcols if '#number' not in x and '#date' not in x and '#img' not in x and '#href' not in x and '#link' not in x]

# Left panel
left_text = pn.Row("####Select Variables for NEMO Processing", margin=(0,0,-15,270))
binary_selector = pn.widgets.CrossSelector(options=varcols, width=630)
left_panel = pn.Column(left_text, binary_selector, css_classes=['widget-box'], margin=(0,30,0,0))

remap_text = pn.pane.Markdown('####      Make selections and run the next cell ', width=650)

# Display widgets
widgets = pn.Row(left_panel)
full_display = pn.Column(widgets,remap_text)
full_display

Column
    [0] Row
        [0] Column(css_classes=['widget-box'], margin=(0, 30, 0, 0))
            [0] Row(margin=(0, 0, -15, 270))
                [0] Markdown(str)
            [1] CrossSelector(options=['Name', 'Affiliation#sort...], width=630)
    [1] Markdown(str, width=650)

In [9]:

# Replace NA with empty in each row
# Convert row to string
# Join row with spaces
concatted = df[binary_selector.value].fillna('').astype(str).dropna().apply(lambda row: ' '.join(row), axis=1)


## 6. Run NEMO

In [11]:
def extract_nemo(payload):
#     print(payload)
    df, respjson = nemo.nemo_annotate(payload)
    return df
extracted_df = concatted.progress_apply(extract_nemo)

  0%|          | 0/1377 [00:00<?, ?it/s]

## 7. Add generated tags to dataframe and explore

In [12]:
extracted_multi_df=pd.DataFrame()
import numpy as np
nan = np.nan
rows=[]
for i in concatted.index:
    print ("###### index == " + str(i))
    rows.append(nemo.create_nemo_dict(extracted_df[i]))

extracted_multi_df = pd.DataFrame(rows)


###### index == 0
###### index == 1
###### index == 2
###### index == 3
###### index == 4
###### index == 5
###### index == 6
###### index == 7
###### index == 8
###### index == 9
###### index == 10
###### index == 11
###### index == 12
###### index == 13
###### index == 14
###### index == 15
###### index == 16
###### index == 17
###### index == 18
###### index == 19
###### index == 20
###### index == 21
###### index == 22
###### index == 23
###### index == 24
###### index == 25
###### index == 26
###### index == 27
###### index == 28
###### index == 29
###### index == 30
###### index == 31
###### index == 32
###### index == 33
###### index == 34
###### index == 35
###### index == 36
###### index == 37
###### index == 38
###### index == 39
###### index == 40
###### index == 41
###### index == 42
###### index == 43
###### index == 44
###### index == 45
###### index == 46
###### index == 47
###### index == 48
###### index == 49
###### index == 50
###### index == 51
###### index == 52
###

###### index == 452
###### index == 453
###### index == 454
###### index == 455
###### index == 456
###### index == 457
###### index == 458
###### index == 459
###### index == 460
###### index == 461
###### index == 462
###### index == 463
###### index == 464
###### index == 465
###### index == 466
###### index == 467
###### index == 468
###### index == 469
###### index == 470
###### index == 471
###### index == 472
###### index == 473
###### index == 474
###### index == 475
###### index == 476
###### index == 477
###### index == 478
###### index == 479
###### index == 480
###### index == 481
###### index == 482
###### index == 483
###### index == 484
###### index == 485
###### index == 486
###### index == 487
###### index == 488
###### index == 489
###### index == 490
###### index == 491
###### index == 492
###### index == 493
###### index == 494
###### index == 495
###### index == 496
###### index == 497
###### index == 498
###### index == 499
###### index == 500
###### index == 501


###### index == 913
###### index == 914
###### index == 915
###### index == 916
###### index == 917
###### index == 918
###### index == 919
###### index == 920
###### index == 921
###### index == 922
###### index == 923
###### index == 924
###### index == 925
###### index == 926
###### index == 927
###### index == 928
###### index == 929
###### index == 930
###### index == 931
###### index == 932
###### index == 933
###### index == 934
###### index == 935
###### index == 936
###### index == 937
###### index == 938
###### index == 939
###### index == 940
###### index == 941
###### index == 942
###### index == 943
###### index == 944
###### index == 945
###### index == 946
###### index == 947
###### index == 948
###### index == 949
###### index == 950
###### index == 951
###### index == 952
###### index == 953
###### index == 954
###### index == 955
###### index == 956
###### index == 957
###### index == 958
###### index == 959
###### index == 960
###### index == 961
###### index == 962


###### index == 1327
###### index == 1328
###### index == 1329
###### index == 1330
###### index == 1331
###### index == 1332
###### index == 1333
###### index == 1334
###### index == 1335
###### index == 1336
###### index == 1337
###### index == 1338
###### index == 1339
###### index == 1340
###### index == 1341
###### index == 1342
###### index == 1343
###### index == 1344
###### index == 1345
###### index == 1346
###### index == 1347
###### index == 1348
###### index == 1349
###### index == 1350
###### index == 1351
###### index == 1352
###### index == 1353
###### index == 1354
###### index == 1355
###### index == 1356
###### index == 1357
###### index == 1358
###### index == 1359
###### index == 1360
###### index == 1361
###### index == 1362
###### index == 1363
###### index == 1364
###### index == 1365
###### index == 1366
###### index == 1367
###### index == 1368
###### index == 1369
###### index == 1370
###### index == 1371
###### index == 1372
###### index == 1373
###### index 

In [13]:
# explore the generated dataframe
panellibs.slider(extracted_multi_df)

ValueError: Slider 'start' and 'end' cannot be equal.

Column(width=800)
    [0] Row
        [0] IntSlider(end=1376, margin=(0, 50, -15, 0), name='Navigate Rows', width=350)
        [1] IntSlider(end=0, margin=(0, 0, 5, 0), name='Navigate Columns', width=350)
    [1] ParamFunction(function, _pane=DataFrame, defer_load=False, width=800)

In [14]:
# alternatively:
with pd.option_context("display.max_columns", None):
    if any("geometry" in col for col in extracted_multi_df.columns):
        display(extracted_multi_df.drop(['geometry'],axis=1))
    else:
        display(extracted_multi_df)
    

""
0
1
2
3
4
...
1372
1373
1374
1375


## 8. Reoder the columns, and decide which columns to keep

In [18]:
# re-order the columns in the df
columnTitles = nemo.column_order()
extracted_multi_df = extracted_multi_df.reindex(columns=columnTitles)

In [19]:
tags_to_keep = pn.widgets.RadioBoxGroup(name='Tags to keep:', options=['Pre-modeled categories: Location, Geopolitical, Organization, Person', 
                                                                        'All internal NEMO output'], 
                                       inline=False)
tags_to_keep

RadioBoxGroup(name='Tags to keep:', options=['Pre-modeled categories: ...], value='Pre-modeled categories: L...)

#### Columns to keep:

        "e_G-y": "Entity: Geopolitical#multi#sortquan",
        "e_G-y_WP": "Entity: Geopolitical URL#hidden",
        "e_O-y": "Entity: Organization#multi#sortquan",
        "e_O-y_WP": "Entity: Organization URL#hidden",
        "e_L-y": "Entity: Location#multi#sortquan",
        "e_L-y_WP": "Entity: Location URL#hidden",
        "e_P-y": "Entity: Personal#multi#sortquan",
        "e_P-y_WP": "Entity: Personal URL#hidden",

        "e_C-y": "Entity: Abstract#multi#sortquan",
        "e_C-y_WP" : "Entity: Abstract URL#hidden",


In [20]:
cols_to_keep = [
     "e_G-y",
    "e_G-y_WP",
    "e_O-y",
    "e_O-y_WP",
    "e_L-y",
    "e_L-y_WP",
    "e_P-y",
    "e_P-y_WP",

    "e_C-y",
    "e_C-y_WP"   
]

if tags_to_keep.value == 'Pre-modeled categories: Location, Geopolitical, Organization, Person':
    extracted_multi_df.drop(extracted_multi_df.columns.difference(cols_to_keep), 1, inplace=True)

panellibs.slider(extracted_multi_df)

TypeError: DataFrame.drop() takes from 1 to 2 positional arguments but 3 positional arguments (and 1 keyword-only argument) were given

In [21]:
# alternatively:
with pd.option_context("display.max_columns", None):
    if any("geometry" in col for col in extracted_multi_df.columns):
        display(extracted_multi_df.drop(['geometry'],axis=1))
    else:
        display(extracted_multi_df)
    

,c_U-n,c_U-y,c_U-y_WP,c_C-y,c_C-y_WP,c_W-y,c_W-y_WP,e_N-y,e_N-y_WP,e_C-y,e_C-y_WP,e_G-y,e_G-y_WP,e_O-y,e_O-y_WP,e_M-y,e_M-y_WP,e_U-y,e_U-y_WP,e_F-y,e_F-y_WP,e_H-y,e_H-y_WP,e_W-y,e_W-y_WP,e_L-y,e_L-y_WP,e_P-y,e_P-y_WP,e_J-y,e_J-y_WP,e_V-y,e_V-y_WP,e_A-y,e_A-y_WP,e_I-y,e_I-y_WP,e_B-y,e_B-y_WP,e_Y-y,e_Y-y_WP,e_S-y,e_S-y_WP,e_E-y,e_E-y_WP,e_K-y,e_K-y_WP,e_R-y,e_R-y_WP,e_Q-y,e_Q-y_WP,e_U-n,e_D-d,d_url-n,d_quantity-n,d_age-n,d_phone-n,d_street-n,d_date-n
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1372,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1373,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1374,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1375,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 9. Rename the columns

In [22]:
# this is a complete list of columns to rename
columns_dict = nemo.columns_dict()

In [23]:
#rename the columns
extracted_multi_df_ren = extracted_multi_df.rename(columns=dict(columns_dict))

In [24]:
df_new = pd.concat([df, extracted_multi_df_ren], axis=1)
print('Dimensions:\n --- The original df: ' +str(df.shape) +'\n --- The NEMO-generated df: '+ str(extracted_multi_df.shape)+'\n --- The concatenated df:' +str(df_new.shape))


Dimensions:
 --- The original df: (1377, 16)
 --- The NEMO-generated df: (1377, 59)
 --- The concatenated df:(1377, 75)


## 10. Only keep columns with more than N records as facets (for new fields, or for all fields), make others #hidden

In [25]:
# set min number of values in a column
mincount = 0

# whether to drop columns from the original dataframe as well
drop_from_original=True

if drop_from_original:
    start_column = 0
else:
    start_column = len(df.columns)

for column in df_new.columns:
    if (df_new.columns.get_loc(column) > start_column) & (df_new[column].count() < mincount):
        if column.find("#multi") > 0:
            new_col = column.split('#')[0]+"#multi#hidden"
        elif column.find("#link") > 0:
            new_col = column
        else:
            new_col = column.split('#')[0]+"#hidden"

        df_new = df_new.rename(columns={column:new_col})
        print(column + " renamed to "+ new_col)
            


## 11. Visualize the generated dataframe

In [26]:
panellibs.slider(df_new)

Column(width=800)
    [0] Row
        [0] IntSlider(end=1376, margin=(0, 50, -15, 0), name='Navigate Rows', width=350)
        [1] IntSlider(end=75, margin=(0, 0, 5, 0), name='Navigate Columns', width=350)
    [1] ParamFunction(function, _pane=DataFrame, defer_load=False, width=800)

In [27]:
# alternatively:
with pd.option_context("display.max_columns", None):
    if any("geometry" in col for col in df_new.columns):
        display(df_new.drop(['geometry'],axis=1))
    else:
        display(df_new)
    

,Name,OAID#link#multi,Affiliation#sortquan,City#sortquan,Region#sortquan,Country#sortquan,Latitude#hidden,Longitude#hidden,Collaborators#multi#link#sortquan,Scope#multi#sortquan,Keywords#multi#sortquan,OA concepts#multi#sortquan,Publications#hidden,Publication Dates#multi#sortquan,#img,#netvis,Concept: Common Phrase#multi#sortquan,Concept: Formal Term#multi#sortquan,Concept: Formal Term URL#hidden,Concept: Abstract#multi#sortquan,Concept: Abstract URL#hidden,Concept W#multi#sortquan,Concept W URL#hidden,Entity: Covid Term#multi#sortquan,Entity: Covid Term URL#hidden,Entity: Abstract#multi#sortquan,Entity: Abstract URL#hidden,Entity: Geopolitical#multi#sortquan,Entity: Geopolitical URL#hidden,Entity: Organization#multi#sortquan,Entity: Organization URL#hidden,Entity: Media#multi#sortquan,Entity: Media URL#hidden,Entity: Formal#multi#sortquan,Entity: Formal URL#hidden,Entity: Facility#multi#sortquan,Entity: Facility URL#hidden,Entity: Holidays#multi#sortquan,Entity: Holidays URL#hidden,Entity: Arts#multi#sortquan,Entity: Arts URL#hidden,Entity: Location#multi#sortquan,Entity: Location URL#hidden,Entity: Personal#multi#sortquan,Entity: Personal URL#hidden,Entity: Computer Term#multi#sortquan,Entity: Computer Term URL#hidden,Entity: Vehicle#multi#sortquan,Entity: Vehicle URL#hidden,Entity: Medical#multi#sortquan,Entity: Medical URL#hidden,Entity: Informatic#multi#sortquan,Entity: Informatic URL#hidden,Entity: Linguistic#multi#sortquan,Entity: Linguistic URL#hidden,Entity: Peoples#multi#sortquan,Entity: Peoples URL#hidden,Entity: Transportation#multi#sortquan,Entity: Transportation URL#hidden,Entity: Event#multi#sortquan,Entity: Event URL#hidden,Entity: Bureaucratic#multi#sortquan,Entity: Bureaucratic URL#hidden,Entity R#multi#sortquan,Entity R URL#hidden,Entity Q#multi#sortquan,Entity Q URL#hidden,Entity: Formal no-URL#multi#sortquan,Entity for Disambiguation#multi#sortquan,Data URL#hidden,Quantity#multi#sortquan,Age#multi#sortquan,Phone#multi#sortquan,Street#multi#sortquan,Date#multi#sortquan
0,A Olioso,https://openalex.org/A4227955457,Unknown,NaN,NaN,NaN,NaN,NaN,https://openalex.org/A4227955454|https://openalex.org/A4227955461|https://openalex.org/A4227955455|https://openalex.org/A4227955463|https://openalex.org/A4227955453|https://openalex.org/A4227955464|https://openalex.org/A4227955456|https://openalex.org/A4227955462|https://openalex.org/A4227955460|https://openalex.org/A4227955459|https://openalex.org/A4227955452|https://openalex.org/A4227955458,aquifer|transboundary,NaN,Groundwater|Geology|Geotechnical engineering|Hydrology (agriculture)|Aquifer|Environmental science|Computer science|Water resource management,"<a href='#' onClick='javascript:getPublication({oaids:""https://openalex.org/A4227955457"",search:""Keywords,Scope"",OAConcepts:""OA concepts""})'>Show publications</a>",2021,US,02ac504b6e11517e2110d174ea70a1a7ac1cf19899e1a0f23c29558f6225db03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A Olioso,https://openalex.org/A4226682424,Unknown,NaN,NaN,NaN,NaN,NaN,https://openalex.org/A4226682420|https://openalex.org/A4226682425|https://openalex.org/A4226682421|https://openalex.org/A4226682429|https://openalex.org/A4226682431|https://openalex.org/A4226682426|https://openalex.org/A4226682422|https://openalex.org/A4226682428|https://openalex.org/A4226682427|https://openalex.org/A4226682419|https://openalex.org/A4226682423|https://openalex.org/A4226682430,aquifer|transboundary,NaN,Groundwater|Geology|Geotechnical engineering|Hydrology (agriculture)|Aquifer|Environmental science|Water resource management,"<a href='#' onClick='javascript:getPublication({oaids:""https://openalex.org/A4226682424"",search:""Keywords,Scope"",OAConcepts:""OA concepts""})'>Show publications</a>",2021,US,8f95a1d08aacc416f1abe22426fe9c9fd2f8f338bb7365407f284e3985165

In [ ]:
# if needed, save to a local file. OPTIONAL
df_to_save = df_new.copy().fillna('')
df_to_save.to_csv('dataframe_after_nemo.csv', index=None)


## 12. Save the new version of CSV file, and give a name to new survey

In [ ]:
if data_select.value == 'Import a local CSV file':
    csv_file = data_input.filename
    dzc_file = ''
    
new_file = suaveint.save_csv_file(df_new, absolutePath, csv_file)

In [ ]:
#Input survey name

import ipywidgets as widgets
from IPython.display import display

input_text = widgets.Text(placeholder='Enter Survey Name...')
output_text = widgets.Text()

def bind_input_to_output(sender):
    output_text.value = input_text.value

# Tell the text input widget to call bind_input_to_output() on submit
input_text.on_submit(bind_input_to_output)

printmd("<b><span style='color:red'>Input survey name here, press Enter, and then run the next cell:</span></b>")
# Display input text box widget for input
display(input_text)

display(output_text)

In [ ]:
#Print survey name
survey_name = output_text.value
printmd("<b><span style='color:red'>Survey Name is: </span></b>" + survey_name)

In [ ]:
suaveint.create_survey(survey_url,new_file, survey_name, dzc_file, user, csv_file, view, views, data_select.value)